In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
from geodatasets import get_path

## Download data##
flight_data = pd.read_csv("/mnt/c/Users/tbrag/OneDrive/Desktop/bda602venv/ML Project/flights_sample_3m.csv")
airport_data = pd.read_csv("/mnt/c/Users/tbrag/OneDrive/Desktop/bda602venv/ML Project/iata-icao.csv")

In [ ]:
##How many airports in the flight data##
print(len(flight_data['ORIGIN'].unique()))
print(len(flight_data['DEST'].unique()))


In [ ]:
###remove uneeded columns in airport data##
airport_data.drop(columns=airport_data.columns.difference(['iata', 'latitude', 'longitude']), inplace=True)
airport_data.head(2)

In [ ]:
# Merge flight_data with airport_data for origin airport
flight_data = flight_data.merge(airport_data, left_on='ORIGIN', right_on='iata', how='left')

# Rename columns to indicate origin airport latitude and longitude
flight_data = flight_data.rename(columns={'latitude': 'origin_latitude', 'longitude': 'origin_longitude'})

# Merge flight_data with airport_data for destination airport
flight_data = flight_data.merge(airport_data, left_on='DEST', right_on='iata', how='left')

# Rename columns to indicate destination airport latitude and longitude
flight_data = flight_data.rename(columns={'latitude': 'dest_latitude', 'longitude': 'dest_longitude'})

# Drop unnecessary columns (iata_x and iata_y)
flight_data.drop(columns=['iata_x', 'iata_y'], inplace=True)

flight_data.head(5)

##export the new table##
flight_data.to_csv('flight_data_with_coordinates.csv', index=False)

In [ ]:
flight_data.head(5)

In [ ]:

# Path to station txt file
station_file = '/mnt/c/Users/tbrag/OneDrive/Desktop/bda602venv/ghcnd-stations (2).txt'

# Define column widths for fixed-width parsing
col_widths = [12, 9, 10, 6, 50]  # Adjust the widths based on your data

# Create a DataFrame from the data
station_df = pd.read_fwf(station_file, widths=col_widths, header=None, names=['Station_ID', 'Latitude', 'Longitude', 'Elevation', 'Station_Name'])

# Display the first 5 rows of the DataFrame
print(station_df.head())

#drop not relevent columns
station_df.drop(columns=['Elevation', 'Station_Name'], inplace=True)
                         
print(station_df.head())


In [ ]:
from shapely.geometry import Point, LineString
import geopandas as gpd
import matplotlib.pyplot as plt
from geodatasets import get_path



# Create Point geometries for origin and destination
#flight_data['origin_geometry'] = flight_data.apply(lambda row: Point(row['origin_longitude'], row['origin_latitude']), axis=1)
#flight_data['dest_geometry'] = flight_data.apply(lambda row: Point(row['dest_longitude'], row['dest_latitude']), axis=1)

# Create LineString geometries
#flight_data['geometry'] = flight_data.apply(lambda row: LineString([row['origin_geometry'], row['dest_geometry']]), axis=1)

# Create GeoDataFrame
#flight_lines_gdf = gpd.GeoDataFrame(flight_data, geometry='geometry')

##export the new table##
#flight_lines_gdf.to_csv('flightpath_lines.csv', index=False)


In [ ]:
###Make station Lat Long into point geometry and create a geopandas dataframe
station_gdf = geopandas.GeoDataFrame(
    station_df, geometry=geopandas.points_from_xy(station_df.Longitude, station_df.Latitude), crs="EPSG:4326"
)
station_gdf.head(10)


In [ ]:
##make flight into geo df using origin Lat Long

flight_gdf = geopandas.GeoDataFrame(
    flight_data, geometry=geopandas.points_from_xy(flight_data.origin_longitude, flight_data.origin_latitude), crs="EPSG:4326"
)
flight_gdf.head(10)


In [ ]:
combined_gdf = flight_gdf.sjoin_nearest(station_gdf,how="inner")



In [ ]:
#removing created right index column
combined_gdf = combined_gdf.drop(columns = "index_right")

#renaming some columns
combined_gdf = combined_gdf.rename(columns = {'geometry':'origin_geometry','Station_ID':'origin_station_id','Latitude':'ori_station_lat','Longitude':'ori_station_long'})

combined_gdf.head(10)


In [ ]:
# Get unique values in the origin station id column
station_values = combined_gdf['origin_station_id'].unique()

# create a list of unique stations to use as a filter later
station_list = list(station_values)

print(station_list)
print(len(station_list))

In [ ]:

# Create a dictionary to map origin coordinates to station IDs
origin_coords_to_id = dict(zip(zip(combined_gdf['origin_latitude'], combined_gdf['origin_longitude']), combined_gdf['origin_station_id']))

# Create a new column 'dest_station_id' based on matching dest coordinates
combined_gdf['dest_station_id'] = list(map(lambda coord: origin_coords_to_id.get(coord), zip(combined_gdf['dest_latitude'], combined_gdf['dest_longitude'])))


combined_gdf.head(10)


In [ ]:
print(len(combined_gdf['dest_station_id'].unique()))
print(len(combined_gdf['origin_station_id'].unique()))

print()

In [ ]:
import pandas as pd
import pyarrow.parquet as pq

df =pd.read_parquet("/mnt/c/Users/tbrag/OneDrive/Desktop/bda602venv/5a6b3c9b29514ca58a2f53f8e4521d65_1.snappy.parquet")
df.head(20)

In [ ]:
import pandas as pd
import pyarrow.parquet as pq

def process_parquet_file(parquet_file, prev_file, station_list, new_column_names):
   

    # Read the Parquet file into a Pandas DataFrame
    df = pd.read_parquet(parquet_file)

    # Rename columns
    df = df.rename(columns=new_column_names)

    df = df.drop(columns=["M_FLAG","Q_FLAG","S_FLAG","OBS_TIME"])

    # Filter rows based on ID list
    filtered_df = df[df['station_id'].isin(station_list)]

   # Concatenate with previous DataFrame if it exists
    if prev_file is not None:
        concat_df = pd.concat([prev_file, filtered_df], ignore_index=True)
    else:
        concat_df = filtered_df

    return concat_df




In [ ]:
import os

# Define processed_dfs as an empty list before the loop
processed_prcp_dfs = []

# Iterate through files in the folder
folder_path = "/mnt/c/Users/tbrag/OneDrive/Desktop/bda602venv/ML Project/weather parquet files/PRCP"
for file_name in os.listdir(folder_path):
    if file_name.endswith(".parquet"):
        file_path = os.path.join(folder_path, file_name)
        print(f"Processing file: {file_path}")
        # Process each file and append the result to processed_dfs
        result_df = process_parquet_file(file_path, None, station_list, {"ID":"station_id","DATA_VALUE":"PCRP"})
        processed_prcp_dfs.append(result_df)
        print(f"Number of rows in result_df: {len(result_df)}")

# Concatenate all processed DataFrames into a single DataFrame
prcp_df = pd.concat(processed_prcp_dfs, ignore_index=True)

# Display the first 10 rows of the concatenated DataFrame
print(prcp_df.head(10))


In [ ]:
# Define processed_dfs for AWND as an empty list before the loop
processed_awnd_dfs = []

# Iterate through files in the folder
folder_path = "/mnt/c/Users/tbrag/OneDrive/Desktop/bda602venv/ML Project/weather parquet files/AWND"
for file_name in os.listdir(folder_path):
    if file_name.endswith(".parquet"):
        file_path = os.path.join(folder_path, file_name)
        print(f"Processing file: {file_path}")
        # Process each file and append the result to processed_dfs
        result_df = process_parquet_file(file_path, None, station_list, {"ID":"station_id","DATA_VALUE":"AWND"})
        processed_awnd_dfs.append(result_df)
        print(f"Number of rows in result_df: {len(result_df)}")

# Concatenate all processed DataFrames into a single DataFrame
awnd_df = pd.concat(processed_awnd_dfs, ignore_index=True)

In [ ]:
# Define processed_dfs for snow as an empty list before the loop
processed_snow_dfs = []

# Iterate through files in the folder
folder_path = "/mnt/c/Users/tbrag/OneDrive/Desktop/bda602venv/ML Project/weather parquet files/SNOW"
for file_name in os.listdir(folder_path):
    if file_name.endswith(".parquet"):
        file_path = os.path.join(folder_path, file_name)
        print(f"Processing file: {file_path}")
        # Process each file and append the result to processed_dfs
        result_df = process_parquet_file(file_path, None, station_list, {"ID":"station_id","DATA_VALUE":"SNOW"})
        processed_snow_dfs.append(result_df)
        print(f"Number of rows in result_df: {len(result_df)}")

# Concatenate all processed DataFrames into a single DataFrame
snow_df = pd.concat(processed_snow_dfs, ignore_index=True)

In [ ]:
# Define processed_dfs for snow depth as an empty list before the loop
processed_snwd_dfs = []

# Iterate through files in the folder
folder_path = "/mnt/c/Users/tbrag/OneDrive/Desktop/bda602venv/ML Project/weather parquet files/SNWD"
for file_name in os.listdir(folder_path):
    if file_name.endswith(".parquet"):
        file_path = os.path.join(folder_path, file_name)
        print(f"Processing file: {file_path}")
        # Process each file and append the result to processed_dfs
        result_df = process_parquet_file(file_path, None, station_list, {"ID":"station_id","DATA_VALUE":"SNWD"})
        processed_snwd_dfs.append(result_df)
        print(f"Number of rows in result_df: {len(result_df)}")

# Concatenate all processed DataFrames into a single DataFrame
snwd_df = pd.concat(processed_snwd_dfs, ignore_index=True)

In [ ]:
# Define processed_dfs for temperature max as an empty list before the loop
processed_tmax_dfs = []

# Iterate through files in the folder
folder_path = "/mnt/c/Users/tbrag/OneDrive/Desktop/bda602venv/ML Project/weather parquet files/TMAX"
for file_name in os.listdir(folder_path):
    if file_name.endswith(".parquet"):
        file_path = os.path.join(folder_path, file_name)
        print(f"Processing file: {file_path}")
        # Process each file and append the result to processed_dfs
        result_df = process_parquet_file(file_path, None, station_list, {"ID":"station_id","DATA_VALUE":"TMAX"})
        processed_tmax_dfs.append(result_df)
        print(f"Number of rows in result_df: {len(result_df)}")

# Concatenate all processed DataFrames into a single DataFrame
tmax_df = pd.concat(processed_tmax_dfs, ignore_index=True)

In [ ]:
# Define processed_dfs for temperature min as an empty list before the loop
processed_tmin_dfs = []

# Iterate through files in the folder
folder_path = "/mnt/c/Users/tbrag/OneDrive/Desktop/bda602venv/ML Project/weather parquet files/TMIN"
for file_name in os.listdir(folder_path):
    if file_name.endswith(".parquet"):
        file_path = os.path.join(folder_path, file_name)
        print(f"Processing file: {file_path}")
        # Process each file and append the result to processed_dfs
        result_df = process_parquet_file(file_path, None, station_list, {"ID":"station_id","DATA_VALUE":"TMIN"})
        processed_tmin_dfs.append(result_df)
        print(f"Number of rows in result_df: {len(result_df)}")

# Concatenate all processed DataFrames into a single DataFrame
tmin_df = pd.concat(processed_tmin_dfs, ignore_index=True)

In [ ]:

# Merge tmin_df with tmax_df on station_id and DATE
weather_df = pd.merge(tmin_df, tmax_df, on=['station_id', 'DATE'])

# Merge merged_df with snow_df on station_id and DATE
weather_df = pd.merge(weather_df , snow_df, on=['station_id', 'DATE'])

# Merge merged_df with snwd_df on station_id and DATE
weather_df = pd.merge(weather_df , snwd_df, on=['station_id', 'DATE'])

# Merge merged_df with awnd_df on station_id and DATE
weather_df  = pd.merge(weather_df , awnd_df, on=['station_id', 'DATE'])

# Merge merged_df with prcp_df on station_id and DATE
weather_df = pd.merge(weather_df , prcp_df, on=['station_id', 'DATE'])

##export the new table##
weather_df.to_csv('weather_data.csv', index=False)
weather_df.head(10)


In [ ]:
if 'GQW00041415' in station_list:
    print("yes in the station_list")
else:
    print("no not in the station_list")

In [ ]:
##change date in weather table to datetime object
weather_df["DATE"] = pd.to_datetime(weather_df["DATE"], format='%Y%m%d')

##export the new table##
weather_df.to_csv('weather_data.csv', index=False)

len(weather["DATE"].unique())


In [ ]:
##change flight date in combined_gdf to date time object
combined_gdf["FL_DATE"] = pd.to_datetime(combined_gdf["FL_DATE"])

##export the new table##
combined_gdf.to_csv('flight_with_station_data.csv', index=False)

In [ ]:
import dask.dataframe as dd

# Convert Pandas DataFrames to Dask DataFrames
combined_gdf__dd = dd.from_pandas(combined_gdf, npartitions=4)  
weather_df_dd = dd.from_pandas(weather_df, npartitions=4)  

# Merge flight_station_data with weather for origin airport
flight_weather_data = combined_gdf_dd.merge(weather_df_dd, left_on=['origin_station_id', "FL_DATE"],
                                           right_on=['station_id', 'DATE'], how='left')

# Rename columns to indicate origin airport weather
flight_weather_data = flight_weather_data.rename(columns={'AWND': 'ori_AWND', 'PRCP': 'ori_PRCP',
                                                          'SNOW': 'ori_SNOW', 'SNWD': 'ori_SNWD',
                                                          'TMAX': 'ori_TMAX', 'TMIN': 'ori_TMIN'})

# Merge flight_station_data with weather_data for destination weather
flight_weather_data = flight_weather_data.merge(weather_df_dd, left_on=['dest_station_id', 'FL_DATE'],
                                               right_on=['station_id', 'DATE'], how='left')

# Rename columns to indicate destination airport weather
flight_weather_data = flight_weather_data.rename(columns={'AWND': 'dest_AWND', 'PRCP': 'dest_PRCP',
                                                          'SNOW': 'dest_SNOW', 'SNWD': 'dest_SNWD',
                                                          'TMAX': 'dest_TMAX', 'TMIN': 'dest_TMIN'})

# Persist the Dask DataFrame if needed for further computations
flight_weather_data = flight_weather_data.persist()

# Compute the Dask DataFrame to get the actual data (if necessary)
flight_weather_data = flight_weather_data.compute()

flight_weather_data.head(10)

In [ ]:
# Merge flight_data with airport_data for origin airport
flight_weather_data = combined_gdf.merge(weather_df, left_on=['origin_station_id',"FL_DATE"], right_on=['station_id','DATE'], how='left')

# Rename columns to indicate origin airport latitude and longitude
flight_weather_data = flight_weather_data.rename(columns={'AWND': 'ori_AWND', 'PRCP': 'ori_PRCP','SNOW':'ori_SNOW','SNWD':'ori_SNWD','TMAX':'ori_TMAX','TMIN':'ori_TMIN'})

# Merge flight_data with airport_data for destination airport
flight_weather_data = flight_weather_data.merge(weather_df, left_on=['dest_station_id','FL_DATE'], right_on=['station_id','DATE'], how='left')

# Rename columns to indicate destination airport latitude and longitude
flight_weather_data = flight_weather_data.rename(columns={'AWND': 'dest_AWND', 'PRCP': 'dest_PRCP','SNOW':'dest_SNOW','SNWD':'dest_SNWD','TMAX':'dest_TMAX','TMIN':'dest_TMIN'})

# Drop unnecessary columns (iata_x and iata_y)
#flight_data.drop(columns=['iata_x', 'iata_y'], inplace=True)

flight_weather_data.head(5)

##export the new table##
#flight_data.to_csv('flight_data_with_coordinates.csv', index=False)